# Лабораторная работа 6

## Сети Кохонена

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import sklearn as sk

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

In [2]:
class Kohonen:
    def __init__(self, dim, nodes_count, lr, nodes = None, r = None):
         self.dim = dim
         self.lr = lr
         self.nodes_count = nodes_count
         
         if r is None:
            self.r = int(nodes_count / 2)
         else:
             self.r = r
         self.nodes = (nodes.copy() if nodes is not None else np.random.rand(nodes_count, dim))


    def fit(self, data, epochs):


        self.masr = []
        self.maslr = []
        
        rcpy = self.r
        lrcpy = self.lr

        for e in range(epochs):
            self.masr.append(self.r)
            self.maslr.append(self.lr)

            np.random.shuffle(data)
            for i in range(len(data)):

                min_dist = np.sqrt(np.sum((data[i] - self.nodes[0]) ** 2))
                index_min_node = 0

                for j in range(self.nodes_count):
                        dist = np.sqrt(np.sum((data[i] - self.nodes[j]) ** 2))
                        if (min_dist > dist):
                            min_dist = dist
                            index_min_node = j

                

                for j in range(self.nodes_count):

                    dist = np.sqrt(np.sum((self.nodes[index_min_node] - self.nodes[j]) ** 2))
                    if  dist < self.r:
                        self.nodes[j] += self.lr * np.exp(- dist / (2 * self.r)) * (- self.nodes[j] + data[i])
            
            self.lr = max(0.01, lrcpy * np.exp(-(e + 1) / epochs))
            self.r = rcpy * np.exp(- (e + 1) * np.log(rcpy) / epochs)

        self.masr = np.array(self.masr)
        self.maslr = np.array(self.maslr)


## Равномерно распределенные точки

In [3]:
data = np.array([[random.random()*100, random.random()*100] for _ in range(100)])
nodes = np.array([[random.random()*100, random.random()*100] for _ in range(5)], dtype = float)

In [4]:
figure = px.scatter(x=data[:, 0], y=data[:, 1])
figure.add_trace(go.Scatter(x=nodes[:, 0], y=nodes[:, 1], mode="markers"))

figure.show()

In [5]:
epochs = 100
kohonen = Kohonen(dim = 2, nodes_count = 5, lr = 1, nodes = nodes, r = 5)
kohonen.fit(data, epochs)

In [6]:
ticker = np.arange(0, epochs, 1)

figure = make_subplots(rows=1, cols=2, subplot_titles=["learning_rate", "radius"])
figure.add_trace(go.Scatter(x=ticker, y=kohonen.maslr, name="learning_rate"), col=1, row=1)
figure.add_trace(go.Scatter(x=ticker, y=kohonen.masr, name="radius"), col=2, row=1)

figure.show()

In [7]:
figure = px.scatter(x=data[:, 0], y=data[:, 1], title="points")
figure.add_trace(go.Scatter(x=kohonen.nodes[:, 0], y=kohonen.nodes[:, 1], mode="markers", name="nodes"))

figure.show()

## Кучки точек

In [8]:
from sklearn.datasets import make_blobs

data_blobs, y = make_blobs(n_samples = 100, centers = 2, n_features = 2, random_state = 1)


nodes_blobs = np.array([[random.random()*10, 0] for _ in range(5)], dtype = float)

figure = plt.figure(figsize = (10, 5))

figure = px.scatter(x=data_blobs[:, 0], y=data_blobs[:, 1], title="points")
figure.add_trace(go.Scatter(x=nodes_blobs[:, 0], y=nodes_blobs[:, 0], mode="markers", name="nodes"))

figure.show()

<Figure size 1000x500 with 0 Axes>

In [9]:
kohonen = Kohonen(dim = 2, nodes_count = 5, lr = 1.5, nodes = nodes_blobs)
kohonen.fit(data_blobs, epochs)

In [10]:
ticker = np.arange(0, epochs, 1)

figure = make_subplots(rows=1, cols=2, subplot_titles=["learning_rate", "radius"])
figure.add_trace(go.Scatter(x=ticker, y=kohonen.maslr, name="learning_rate"), col=1, row=1)
figure.add_trace(go.Scatter(x=ticker, y=kohonen.masr, name="radius"), col=2, row=1)

figure.show()

In [11]:
figure = px.scatter(x=data_blobs[:, 0], y=data_blobs[:, 1], title="points")
figure.add_trace(go.Scatter(x=kohonen.nodes[:, 0], y=kohonen.nodes[:, 1], mode="markers", name="nodes"))

figure.show()